In [304]:
import pandas as pd
import os
import openpyxl
import numpy as np
import re
from datetime import datetime
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
lemmatizer = WordNetLemmatizer()

In [221]:
class FinancialReport():
    def __init__(self,file):
        self.file = file
        self.readFile()
        self.findBalanceSheet()
        self.findCashFlowSheet()
    def readFile(self):
        path = os.path.join("financial statements",self.file)
        self.statements = pd.read_excel(path,sheet_name=None)
    def getSheetNames(self):
        return self.statements.keys()
    def findBalanceSheet(self):
        balanceSheetName = [name for name in self.getSheetNames() if "BALANCE" in name][0]
        self.balance = self.statements[balanceSheetName]
        self.balance.columns = ["label"]+list(self.balance.columns[1:])
    def findCashFlowSheet(self):
        cashflowSheetName = [name for name in self.getSheetNames() if "CASH" in name]
        if len(cashflowSheetName) > 0:
            self.cashflow = self.statements[cashflowSheetName[0]]
        else:
            cashflowSheetName = [name for name in self.getSheetNames() if "CONDENSED" in name][-1]
            self.cashflow = self.statements[cashflowSheetName]
        self.cashflow.columns = self.cashflow.iloc[0]
        self.cashflow = self.cashflow.drop(0)
        self.cashflow.columns = ["label"]+list(self.cashflow.columns[1:])
    def getBalanceSheet(self):
        return self.balance
    def getCashFlowSheet(self):
        return self.cashflow
    

In [463]:
class CashFlow():
    def __init__(self):
        self.finalCashFlow = pd.DataFrame(columns=["title","num_title","label"])
        self.setFinancialStatements()
        self.addDataToCashFlowSheet()
        self.setQuarters()
        self.rearrangeFinalCashFlowSheet()
        
        
    def getFiles(self):
        files = os.listdir("financial statements")
        files.remove(".DS_Store")
        files = [file for file in files if file[0] != "~"]
        return files

    def setFinancialStatements(self):
        self.files = self.getFiles()
        self.FinancialReports = [FinancialReport(file) for file in self.files]
        
    def addDataToCashFlowSheet(self):
        
        for FinancialStatements in self.FinancialReports:
            print(FinancialStatements.file)
            cashflow = FinancialStatements.getCashFlowSheet().copy()
            
            cols = ["label"]+[col for col in cashflow.columns[1:] if col not in self.finalCashFlow.columns] ## remove columns of data already in main balance sheet
            cashflow = cashflow[cols]
            
            cashflow = self.parseCashFlowStatement(cashflow)
            
            self.finalCashFlow = pd.merge(self.finalCashFlow, cashflow, how="outer",on=["label","title"],suffixes=('', '_new'))
            #self.finalCashFlow.title = [self.finalCashFlow.title[i] if not pd.isnull(self.finalCashFlow.title[i]) else self.finalCashFlow.title_new[i] for i in range(len(self.finalCashFlow))] #merge titles
            self.finalCashFlow.num_title = [self.merge_number(self.finalCashFlow.num_title[i],self.finalCashFlow.num_title_new[i]) for i in range(len(self.finalCashFlow))] #merge the number for the titles. use greatest num bc bigger num means there is section before it
            self.finalCashFlow = self.finalCashFlow.drop(["num_title_new"],axis=1)

    def rearrangeFinalCashFlowSheet(self):
        self.finalCashFlow = self.finalCashFlow.replace(" ",np.nan)
        # put Nets at the bottom of each section
        Nets = self.finalCashFlow.loc[self.finalCashFlow.label.str.startswith("net")].reset_index(drop=True) 
        self.finalCashFlow = self.finalCashFlow.loc[~self.finalCashFlow.label.str.startswith("net")].reset_index(drop=True)
        self.finalCashFlow = pd.concat([self.finalCashFlow,Nets]) 
        self.finalCashFlow = self.finalCashFlow.reset_index(drop=True).reset_index().rename(columns={"index":"num_label"})
        self.finalCashFlow = self.finalCashFlow.sort_values(["num_title","num_label"]) # order by title and then location
        self.finalCashFlow = self.finalCashFlow.loc[~self.finalCashFlow[cols].isnull().all(axis=1)]
        #self.finalCashFlow = self.finalCashFlow.drop(["num_title","num_label"],axis=1)
        
    def parseCashFlowStatement(self,balance):
        num = 0
        index = []
        title = False
        for label in balance.label:
            if ":" in label:
                title = re.sub("\([\w\W]+\)","",label[:-1]).replace("  "," ")
                num += 1
                continue
            if title:
                new_label = label.lower()
                new_label = new_label.replace("Gain","Loss")
                new_label = re.sub("\([\w\W]+\)","",new_label).replace("  "," ").strip()
                index.append([title,num,label,new_label])
            
            if "net" == label[:3]:
                title = np.nan
                num += 1
    
        balance = balance.loc[balance.label.isin(np.array(index)[:,2])].copy()
        balance.loc[:,["title","num_title","label"]] = list(np.array(index)[:,[0,1,3]])
        balance = balance.set_index(["title","num_title","label"]).reset_index()
        return balance
    
    def clean_up_string(self,string):
        string = string.strip()
        space_positions = [x.start() for x in re.finditer('\ ',  string)]
        for i in reversed(space_positions):
            if string[i+1:] in ["of", "and"]:
                string = string[:i]
        return string
    
    def merge_number(self,a,b):
        if pd.isnull(a):
            return b
        if pd.isnull(b):
            return a
        return max(a,b)
    
    def getColumnNamesForQuarters(self):
        cols = list(self.finalCashFlow.columns)
        cols = [col for col in cols if col not in ["title","label","num_title"]]
        return cols
        
    def setQuarters(self):
        dates = self.getColumnNamesForQuarters()
        dates.sort(key=lambda date: datetime.strptime(date.replace(".",""), '%b %d, %Y'))
        self.finalCashFlow = self.finalCashFlow[["title","label","num_title"]+dates]
        self.finalCashFlow.columns = ["title","label","num_title"]+[self.parseQuarter(date) for date in self.getColumnNamesForQuarters()]
    
    def parseQuarter(self,date):
        mo = date.split(" ")[0].strip(".")
        year = date.split(", ")[-1]
        if mo in ["May"]:
            return "Q1 "+str(int(year)+1)
        
        if mo in ["Jul","Aug"]:
            return "Q2 "+str(int(year)+1)
        
        if mo in ["Oct","Nov"]:
            return "Q3 "+str(int(year)+1)
        
        if mo in ["Jan","Feb"]:
            return "Q4 "+str(int(year))
        
    def reconcile_title_numbers():
        self.finalCashFlow.groupby("label")["num_title"].max().reset_index()
    

In [464]:
cashflow = CashFlow()
finalCashFlow = cashflow.finalCashFlow
finalCashFlow

q3 2021.xlsx
q3 2020.xlsx
q1 2019.xlsx
q2 2021.xlsx
q2 2020.xlsx
10k 2020 (2019).xlsx
10k 2019 (2020).xlsx
q1 2021.xlsx
q2 2019.xlsx
q1 2020.xlsx
10k 2022 (2021).xlsx
q3 2019.xlsx
10k 2021 (2022).xlsx


,num_label,title,label,num_title,Q4 2017,Q4 2018,Q1 2019,Q2 2019,Q3 2019,Q4 2019,...,Q3 2020,Q4 2020,Q1 2021,Q2 2021,Q3 2021,Q4 2021,Q1 2022,Q2 2022,Q3 2022,Q4 2022
87,87,Cash flows from operating activities,net income,1,676,848,164,461,727,1003,...,535,351,-932,-994,-899,-665,166,424,272,256
0,0,Adjustments to reconcile net income to net cas...,depreciation and amortization,2,593,559,140,280,425,578,...,417,557,130,256,381,507,120,244,372,504
1,1,Adjustments to reconcile net income to net cas...,share-based compensation,2,76,87,21,48,72,91,...,64,68,18,35,55,77,36,72,97,139
2,2,Adjustments to reconcile net income to net cas...,"operating lease, impairment loss",2,NaN,0,NaN,NaN,NaN,0,...,1,239,360,361,361,NaN,5,6,6,NaN
3,3,Adjustments to reconcile net income to net cas...,other asset impairment charges,2,107,28,NaN,NaN,NaN,14,...,9,98,124,127,127,NaN,0,1,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,56,Non-cash investing activities,purchases of property and equipment not yet pa...,6,56,77,NaN,NaN,NaN,93,...,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,79,Supplemental disclosure of cash flow information,cash paid for operating lease liabilities,6,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1096,NaN,NaN,NaN,1061
35,35,Supplemental disclosure of cash flow information,"cash paid for income taxes during the period, ...",7,488,570,19,61,73,143,...,117,176,37,53,8,20,20,147,181,215
45,45,Supplemental disclosure of cash flow information,cash paid for interest during the period,7,82,76,38,NaN,NaN,76,...,NaN,76,NaN,NaN,NaN,145,NaN,NaN,NaN,180


In [465]:
finalCashFlow.to_csv("finalCashFlow.csv")

In [442]:




# cols = finalCashFlow.columns[4:]
# groupby = {x:"sum" for x in cols}
# groupby.update({"label":"first","num_label":"min","num_title":"min"})
# finalCashFlow[cols] = finalCashFlow[cols].fillna(0).astype('int32')
# labels = [x.replace("Gain","Loss") for x in finalCashFlow.label]
# labels = [re.sub("\([\w\W]+\)","",x).replace("  "," ").strip() for x in labels]

# finalCashFlow["new_label"] = labels
#set up num label
# finalCashFlow = finalCashFlow.groupby(["title","new_label"]).aggregate(groupby).reset_index()
# finalCashFlow = finalCashFlow.sort_values(["num_title","num_label"])


# labels = [pos_tag(x.split()) for x in labels]

# dict_pos = {"J": wordnet.ADJ,
#                 "NN": wordnet.NOUN,
#                 "VBD": wordnet.VERB,
#                 "VBZ": wordnet.VERB,
#                 "VBG": wordnet.VERB,
#                 "VBN": wordnet.VERB,
#                 "RB": wordnet.ADV,
#                 'IN':False}
# def pos(x):
#     return x.lower()[0]
# labels = map(lambda sent: list(map(lambda x: [x[0].lower(),dict_pos.get(x[1],wordnet.NOUN)], sent)),labels)
# labels = [[lemmatizer.lemmatize(word[0],pos=word[1]) for word in sent if word[1]] for sent in labels ]
# [" ".join(sent) for sent in labels]